In [2]:
# Imports required tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

In [3]:
# Uploads the dataset from excel
from google.colab import files
uploaded = files.upload()

Saving TrainingData.xlsx to TrainingData.xlsx


In [4]:
# Reads the dataset
df = pd.read_excel('/content/TrainingData.xlsx', sheet_name = 'RandomClass')
df.head()

,Tissue thickness (mm),Tissue stiffness (MPa),Cutting force (N),Class
0,15.063069,0.008795,1.23,Fat
1,32.902929,0.000981,32.87,Muscle
2,1.027028,0.221932,47.19,Skin
3,18.068082,0.004617,18.18,Muscle
4,1.369371,0.015732,49.98,Skin


In [5]:
# Properties of the data
print(df.shape)
num_of_classes = len(df.Class.unique())
print(num_of_classes)

df.describe()

(440, 4)
3


,Tissue thickness (mm),Tissue stiffness (MPa),Cutting force (N)
count,440.000000,440.000000,440.000000
mean,21.965642,0.061888,32.519432
std,19.697349,0.220711,23.087394
min,0.865366,-0.210942,0.010000
25%,2.282275,0.001398,2.027500
50%,15.652637,0.004027,38.820000
75%,34.704993,0.021314,52.780000
max,85.081802,2.092219,84.540000


In [6]:
# Here we define the input and output
X = df.drop(axis = 0, columns = ['Class'])
Y = df.Class

print(X.shape)
print(Y.shape)

(440, 3)
(440,)


In [7]:
# Here we split the training and test data with a ratio of 67:33
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)

In [8]:
from xgboost import XGBClassifier

# Create a classifier
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', random_state=42, eval_metric="auc", num_class=num_of_classes)

# Fit the classifier with the training data
xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='auc',
              gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, num_class=3, objective='multi:softprob',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=1, verbosity=1)

In [9]:
# Using ROC AUC to evaluate the model
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing

# The trained model is used to predict the output of the test data
pred = xgb.predict(X_test)

lb = preprocessing.LabelBinarizer()
lb.fit(y_test)

y_test_lb = lb.transform(y_test)
pred_lb = lb.transform(pred)

roc_auc_score(y_test_lb, pred_lb, average = 'macro')

1.0

In [10]:
# The model performs with 100% accuracy

In [11]:
output = pd.DataFrame()
output['Expected Output'] = y_test
output['Predicted Output'] = pred
output.head()

,Expected Output,Predicted Output
265,Muscle,Muscle
78,Muscle,Muscle
347,Muscle,Muscle
255,Skin,Skin
327,Muscle,Muscle


### Let's test our model on unlabelled data

In [12]:
ULdata = pd.read_excel('/content/TrainingData.xlsx', sheet_name = 'UL')

In [13]:
ULdata

,Tissue thickness (mm),Tissue stiffness (MPa),Cutting force (N)
0,1.654657,0.032625,52.316021
1,0.903403,0.217328,46.184685
2,1.188688,0.167570,48.513027
3,0.950950,0.407567,46.572739
4,1.741660,0.031438,53.026096
...,...,...,...
1159,45.664518,0.001882,45.511856
1160,42.602466,0.001849,42.479420
1161,42.336200,0.002785,42.215733
1162,57.912727,0.002249,57.641621


In [14]:
y_prediction = xgb.predict(ULdata)
y_prediction

array(['Skin', 'Skin', 'Skin', ..., 'Muscle', 'Muscle', 'Muscle'],
      dtype=object)

In [17]:
import numpy as np
np.savetxt('MultiClassPrediction.csv', y_prediction, fmt='%s')